### Import Libraries

In [3]:
import pandas as pd
import numpy as np
import h5py
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import os
import string as st
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
# Load spaCy's English model
nlp = spacy.load('en_core_web_sm')

## Function

In [4]:
def preprocess_text(text):
    doc = nlp(text.lower().strip())  # Lowercase and remove whitespace
    
    # Remove stopwords, punctuation, and lemmatize
    tokens = [token.lemma_ for token in doc 
             if not token.is_stop and not token.is_punct and token.is_alpha]
    
    return ' '.join(tokens)

In [5]:
def extract_linguistic_features(text):
    doc = nlp(text)
    features = {
        'num_verbs': len([token for token in doc if token.pos_ == 'VERB']),
        'num_nouns': len([token for token in doc if token.pos_ == 'NOUN']),
        'sentence_length': len(doc),
        'blooms_verb_present': any(token.text in {'analyze', 'evaluate', 'create'} for token in doc)
    }
    return features

## Pre-Processing

In [7]:
# Load dataset
df = pd.read_csv(os.getcwd().replace('notebook' , 'dataset') + '/question_dataset.csv')

# Apply preprocessing
df['processed_question'] = df['QUESTION'].apply(preprocess_text)

In [8]:
# Load pre-trained BERT (TensorFlow version)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

# Tokenize and encode
inputs = tokenizer(df['processed_question'], return_tensors='pt', padding=True, truncation=True)
outputs = model(**inputs)

# Extract embeddings (CLS token for sentence representation)
sentence_embedding = outputs.last_hidden_state[:, 0, :]
print("BERT Sentence Embedding Shape:", sentence_embedding.shape)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [7]:
ling_features = df['processed_question'].apply(extract_linguistic_features).apply(pd.Series)
X_features = pd.concat([ling_features, pd.DataFrame(X_tfidf.toarray())], axis=1)